# Generate molecules

In [1]:
# Imports

from hyformer.configs.tokenizer import TokenizerConfig
from hyformer.configs.model import ModelConfig

from hyformer.utils.tokenizers.auto import AutoTokenizer
from hyformer.models.auto import AutoModel

from hyformer.utils.chemistry import is_valid


# auxiliary imports
import torch
import torch.nn.functional as F

# autoreload magic
%reload_ext autoreload
%autoreload 2


In [35]:
# Paths

MODEL_NAME = 'hyformer_small'
TASK_NAME = 'lm'

TOKENIZER_CONFIG_PATH = 'configs/tokenizers/smiles/config.json'
MODEL_CONFIG_PATH = 'configs/models/hyformer_small/config.json'
MODEL_CHECKPOINT_PATH = f'/lustre/groups/aih/hyformer/results/distribution_learning/guacamol/hyformer_small/lm_enumerated/checkpoint.pt'

In [42]:
# Load model

tokenizer_config = TokenizerConfig.from_config_filepath(TOKENIZER_CONFIG_PATH)
tokenizer = AutoTokenizer.from_config(tokenizer_config)

# Load model
model_config = ModelConfig.from_config_filepath(MODEL_CONFIG_PATH)
model = AutoModel.from_config(model_config)
model.load_pretrained(filepath=MODEL_CHECKPOINT_PATH)




In [43]:
batch_size = 64
temperature = 1.2
top_k = None
top_p = 0.9
max_sequence_length = 100

device = 'cuda:0'

generator = model.to_generator(
    tokenizer=tokenizer,
    batch_size=batch_size,
    temperature=temperature,
    top_k=top_k,
    top_p=top_p,
    max_sequence_length=max_sequence_length,
    device=device,
    use_cache=False
    )

samples = generator.generate(number_samples=1000)
is_valid_smiles = [is_valid(sample) for sample in samples]
print("Validity: ", sum(is_valid_smiles) / len(is_valid_smiles))

Generating samples: 100%|██████████| 16/16 [00:15<00:00,  1.01it/s]

Validity:  0.995


In [51]:
len(list(set(samples))) / len(samples)

1.0

## MolGPT

In [3]:
# Paths

MODEL_NAME = 'molgpt'

MODEL_CONFIG_PATH = f'configs/models/{MODEL_NAME}/config.json'
MODEL_CHECKPOINT_PATH = f'/lustre/groups/aih/hyformer/results/distribution_learning/guacamol/molgpt/guacamol_nocond.pt'

In [23]:
# Load dataset

from hyformer.models.baselines.molgpt import SmilesDataset

dataset = SmilesDataset(['C'])

# Load model
tokenizer = None

# Load model
model_config = ModelConfig.from_config_filepath(MODEL_CONFIG_PATH)
model = AutoModel.from_config(model_config)
model.load_pretrained(filepath=MODEL_CHECKPOINT_PATH)
model._model.to('cuda')




Filtering data: 100%|██████████| 1/1 [00:00<00:00, 1608.25it/s]


Removed key: prop_nn.weight from checkpoint; not found in model or size mismatch.
Removed key: prop_nn.bias from checkpoint; not found in model or size mismatch.


GPT(
  (tok_emb): Embedding(94, 256)
  (type_emb): Embedding(2, 256)
  (drop): Dropout(p=0.1, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (ln1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (attn): CausalSelfAttention(
        (key): Linear(in_features=256, out_features=256, bias=True)
        (query): Linear(in_features=256, out_features=256, bias=True)
        (value): Linear(in_features=256, out_features=256, bias=True)
        (attn_drop): Dropout(p=0.1, inplace=False)
        (resid_drop): Dropout(p=0.1, inplace=False)
        (proj): Linear(in_features=256, out_features=256, bias=True)
      )
      (mlp): Sequential(
        (0): Linear(in_features=256, out_features=1024, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=1024, out_features=256, bias=True)
        (3): Dropout(p=0.1, inplace=False)
      )
    )
    (1): Block(
      (ln1): LayerNorm((

In [24]:

batch_size = 2
x = torch.tensor([dataset.stoi[s] for s in dataset.regex.findall('C')], dtype=torch.long)[None,...].repeat(batch_size, 1).to('cuda')

In [25]:
y = model.sample(x=x, steps=model._config.block_size, temperature=0.7, sample=True, top_k=None, prop = None, scaffold = None)



ValueError: too many values to unpack (expected 3)

In [ ]:

for gen_mol in y:
        completion = ''.join([itos[int(i)] for i in gen_mol])
        completion = completion.replace('<', '')
        # gen_smiles.append(completion)
        mol = get_mol(completion)
        if mol:
                molecules.append(mol)